# Webscraping Energy_Info with SQL Server Connection

## Webscraping

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import datetime

In [62]:
# power consumption in Germany on one specific day
siteurl = "https://api.energy-charts.info/total_power?country=de&start=2023-01-01T00%3A00%2B01%3A00&end=2023-01-01T23%3A45%2B01%3A00"
response = requests.get(siteurl)
power_consumption = json.loads(response.content.decode('utf-8'))

# create a list of the time_stamps, types and data
unix_seconds_list = []
name_list = []
data_list = []
for typ in range(len(power_consumption['production_types'])):
    for time in range(len(power_consumption['unix_seconds'])):
        # append unix_seconds
        unix_seconds_list.append(datetime.datetime.fromtimestamp(power_consumption['unix_seconds'][time]).strftime("%m/%d/%Y, %H:%M:%S"))
        # append production_types
        name_list.append(power_consumption['production_types'][typ]['name'])
        # append data
        data_list.append(power_consumption['production_types'][typ]['data'][time])

# create dict
power_consumption_dict = {'unix_seconds' : unix_seconds_list, 'name' : name_list, 'data' : data_list}

In [63]:
print(unix_seconds_list)

['12/31/2022, 23:00:00', '12/31/2022, 23:15:00', '12/31/2022, 23:30:00', '12/31/2022, 23:45:00', '01/01/2023, 00:00:00', '01/01/2023, 00:15:00', '01/01/2023, 00:30:00', '01/01/2023, 00:45:00', '01/01/2023, 01:00:00', '01/01/2023, 01:15:00', '01/01/2023, 01:30:00', '01/01/2023, 01:45:00', '01/01/2023, 02:00:00', '01/01/2023, 02:15:00', '01/01/2023, 02:30:00', '01/01/2023, 02:45:00', '01/01/2023, 03:00:00', '01/01/2023, 03:15:00', '01/01/2023, 03:30:00', '01/01/2023, 03:45:00', '01/01/2023, 04:00:00', '01/01/2023, 04:15:00', '01/01/2023, 04:30:00', '01/01/2023, 04:45:00', '01/01/2023, 05:00:00', '01/01/2023, 05:15:00', '01/01/2023, 05:30:00', '01/01/2023, 05:45:00', '01/01/2023, 06:00:00', '01/01/2023, 06:15:00', '01/01/2023, 06:30:00', '01/01/2023, 06:45:00', '01/01/2023, 07:00:00', '01/01/2023, 07:15:00', '01/01/2023, 07:30:00', '01/01/2023, 07:45:00', '01/01/2023, 08:00:00', '01/01/2023, 08:15:00', '01/01/2023, 08:30:00', '01/01/2023, 08:45:00', '01/01/2023, 09:00:00', '01/01/2023, 09

## Data to SQL Server

In [7]:
import pyodbc
from config import database_config

In [65]:
server = database_config['server']
database = database_config['database']
username = database_config['username']
password = database_config['password']
driver= database_config['driver']

try:
    with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
        with conn.cursor() as cursor:
            for i in range(len(power_consumption_dict['unix_seconds'])):
                unix_seconds = power_consumption_dict['unix_seconds'][i]
                name = power_consumption_dict['name'][i]
                data = power_consumption_dict['data'][i]

                # INSERT-Befehl ausführen
                query = "INSERT INTO dbo.EnergyCharts ([unix_seconds], [name], [data]) VALUES (?, ?, ?)"
                cursor.execute(query, (unix_seconds, name, data))
except pyodbc.Error as ex:
    print("Fehler beim Verbinden zur Datenbank:", ex)